## 五、卷积神经网络（LeNet）
### 5.1 LeNet模型
    LeNet 分为卷积层块和全连接层块两个部分。
    卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，例如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。
    卷积层块由两个这样的基本单位重复堆叠构成。
    在卷积层块中，每个卷积层都使用 5×5 的窗口，并在输出上使用 sigmoid 激活函数。
    第一个卷积层输出通道数为 6，第二个卷积层输出通道数则增加到 16。
    这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。
    卷积层块的两个最大池化层的窗口形状均为 2×2，且步幅为 2。
    由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。
    
    卷积层块的输出形状为（批量大小，通道，高，宽）。
    当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。
    也就是说，全连接层的输入形状将变成二维，其中第一维为小批量中的样本，第二维为每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。
    全连接层块含三个全连接层。它们的输出个数分别是 120、84 和 10。其中 10 为输出的类别个数。

In [2]:
import gluonbook as gb
import mxnet as mx
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import loss as gloss,nn
import time

net=nn.Sequential()
net.add(nn.Conv2D(channels=6,kernel_size=5,activation='sigmoid'),
        nn.MaxPool2D(pool_size=2,strides=2),
        nn.Conv2D(channels=16,kernel_size=5,activation='sigmoid'),
        nn.MaxPool2D(pool_size=2,strides=2),
        # Dense 会默认将（批量大小，通道，高，宽）形状的输入转换成批量大小，通道 * 高 * 宽）形状的输入。
        nn.Dense(120,activation='sigmoid'),
        nn.Dense(84,activation='sigmoid'),
        nn.Dense(10))

In [3]:
X=nd.random.uniform(shape=(1,1,28,28))
net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:\t',X.shape)

conv0 output shape:	 (1, 6, 24, 24)
pool0 output shape:	 (1, 6, 12, 12)
conv1 output shape:	 (1, 16, 8, 8)
pool1 output shape:	 (1, 16, 4, 4)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)


### 5.2 获取数据和训练

In [8]:
batch_size=256
train_iter,test_iter=gb.load_data_fashion_mnist(batch_size=batch_size)

In [9]:
# 测试机器是够可以使用gpu

def try_gpu4():
    try:
        ctx=mx.gpu()
        _ =nd.zeros((1,),ctx=ctx)
    except mx.base.MXNetError:
        ctx=mx.cpu()
    return ctx
ctx=try_gpu4()
ctx

cpu(0)

In [ ]:
def evaluate_ch5(net,train_iter,te)